#  Практика.


In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss
from typing import Callable
import pandas as pd

In [2]:
df = pd.read_csv('hw_6_data.txt', sep=' ', names=['first', 'second', 'third', 'winning', 'reporter?'])
df.head()

,first,second,third,winning,reporter?
0,2,0,0,0,1
1,0,5,1,0,1
2,0,0,0,0,1
3,2,0,0,0,1
4,0,0,0,0,1


In [3]:
df = df[df['reporter?'] == 1]

In [4]:
df['reporter?'].mean()

1.0

In [5]:
df = df - (df > 4)
df.head()

,first,second,third,winning,reporter?
0,2,0,0,0,1
1,0,4,1,0,1
2,0,0,0,0,1
3,2,0,0,0,1
4,0,0,0,0,1


In [37]:
def correction(df, name):
    ret = np.unique(df[name], return_counts = True)[1]
    if name == 'second': 
        ret = np.append(ret, ret[5])
        ret[5] = 0
    return ret / 138

In [45]:
names = ['first', 'second', 'third']
model = np.vstack([correction(df, name) for name in names])
model

array([[0.42753623, 0.35507246, 0.10144928, 0.04347826, 0.02173913,
        0.00724638, 0.04347826],
       [0.61594203, 0.05797101, 0.17391304, 0.11594203, 0.00724638,
        0.        , 0.02898551],
       [0.55797101, 0.2826087 , 0.04347826, 0.00724638, 0.03623188,
        0.02173913, 0.05072464]])

In [8]:
def likehood(theta: pd.DataFrame, model) -> float:
    return -np.sum(np.log(theta) * model)

In [9]:
expected = np.array([1 / 2, 1 / 4, 1 / 8, 1 / 16, 1 /64, 1 / 64, 1 / 32])
theta = np.array([expected,
                 expected,
                 expected])
theta

array([[0.5     , 0.25    , 0.125   , 0.0625  , 0.015625, 0.015625,
        0.03125 ],
       [0.5     , 0.25    , 0.125   , 0.0625  , 0.015625, 0.015625,
        0.03125 ],
       [0.5     , 0.25    , 0.125   , 0.0625  , 0.015625, 0.015625,
        0.03125 ]])

##  Ограничения.

 * $\theta \geq 0$
 .$0 \leq I_{21} \dot \theta \leq \text{np.inf}$
 * $\sum_i p_i^j = 1$.
 1 $\leq (111100000000\\
                               000011110000\\
                               000000001111)\theta \leq 1 $
 * $\mathbb E \text{reward} $.
 $\mathbb E = p^3 \otimes (p^2 \otimes (p^1 \otimes M))$
 
 Надо еще найти $\theta_0$, чутка уменьшить вероятность 0 и увеличить вероянтость алмазика, пока $\mathbb E \ne 0.92$

In [10]:
def winning(a, b, c):
    cs = [0, 0, 0, 0, 0, 0, 0]
    cs[a] += 1
    cs[b] += 1
    cs[c] += 1
    trips = [0, 10, 25, 40, 800, 10, 80]
    for i, t in enumerate(cs):
        if t == 3: return trips[i]
    if sum(cs[1:4]) == 3: return 5
    if cs[5] == 2 and cs[4] == 0: return 5
    if cs[5] == 1 and cs[4] == 0: return 2
    if cs[4] == 1:
        for i, t in enumerate(cs):
            if t == 2: return 2 * trips[i]
        if cs[5] == 1: return 10
        if sum(cs[1:4]) == 2: return 10
    if cs[4] == 2:
        for i, t in enumerate(cs):
            if t == 1: return 4 * trips[i]
    return 0

In [11]:
from scipy.optimize import bisect

In [39]:
def cheat(cx, model):
    model[1][0] -= cx
    model[1][4] += cx
    e = 0
    for a in range(7):
        for b in range(7):
            for c in range(7):
                e += winning(a, b, c) * model[0][a] * model[1][b] * model[2][c]
    return e - 0.92

c = bisect(cheat, 0, 1, args = model)

In [46]:
model[1][0] -= c
model[1][4] += c
print(np.sum(model, axis=1))
print(model)

[1. 1. 1.]
[[0.42753623 0.35507246 0.10144928 0.04347826 0.02173913 0.00724638
  0.04347826]
 [0.61594203 0.05797101 0.17391304 0.11594203 0.00724638 0.
  0.02898551]
 [0.55797101 0.2826087  0.04347826 0.00724638 0.03623188 0.02173913
  0.05072464]]


In [21]:
from scipy.optimize import minimize

In [41]:
def jacob(x, model):
    return -model / x

In [56]:
def hessi(x, model):
    return np.diag(model / x ** 2)
    

In [47]:
model = np.hstack(model)
model

array([0.42753623, 0.35507246, 0.10144928, 0.04347826, 0.02173913,
       0.00724638, 0.04347826, 0.61594203, 0.05797101, 0.17391304,
       0.11594203, 0.00724638, 0.        , 0.02898551, 0.55797101,
       0.2826087 , 0.04347826, 0.00724638, 0.03623188, 0.02173913,
       0.05072464])

In [22]:
from scipy.optimize import LinearConstraint, NonlinearConstraint

In [55]:
ret = minimize(likehood, model, args=model,
              jac = jacob,
              hess = hessi,
              constraints = [
                  {
                      'type': 'ineq', 'fun': LinearConstraint()
                  }
              ]
              )

D:\coding\Conda\lib\site-packages\scipy\optimize\_minimize.py:524: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,
<ipython-input-8-1a5411671a2e>:2: RuntimeWarning: divide by zero encountered in log
  return -np.sum(np.log(theta) * model)
<ipython-input-8-1a5411671a2e>:2: RuntimeWarning: invalid value encountered in multiply
  return -np.sum(np.log(theta) * model)
<ipython-input-41-a171c7398d1f>:2: RuntimeWarning: invalid value encountered in true_divide
  return -model / x


# Домашка.

## 1.

In [2]:
brain = [1380, 1344, 1356, 1291, 1308, 1271, 1371, 1430,
         1381, 1457, 1492, 1240, 1256, 1466, 1214, 1448,
         1510, 1395, 1507, 1264, 1293, 1251, 1380, 1386,
         1411, 1434, 1302, 1529, 1352, 1494, 1348, 1464,
         1286, 1345, 1491, 1259, 1541, 1214, 1310, 1286]

In [4]:
ss.norm.cdf(np.mean(brain), loc = 1400, scale = np.std(brain, ddof=1) / np.sqrt(len(brain)))

0.018568470027286878

In [5]:
np.mean(brain)

1368.925

In [111]:
ss.ttest_1samp(brain, 1400, alternative='less')

Ttest_1sampResult(statistic=-2.0842554201754617, pvalue=0.02186741366047351)

In [112]:
t = np.sqrt(len(brain)) * (np.mean(brain) - 1400) / np.std(brain, ddof=1)

ss.t.cdf(t, df=len(brain) - 1)

0.02186741366047351

## 2.

In [113]:
def likehood(P):
    return -sum(np.log(P))

In [128]:
def jac(P):
    return -1/P

In [129]:
def hes(P):
    return np.diag(1 / P ** 2)

In [130]:
ret = minimize(likehood, np.ones(n) / n, jac=jac, hess=hes,
              constraints = [
                  LinearConstraint(np.ones(n), 1, 1),
                  LinearConstraint(np.array(brain), 1400, 1400),
                  LinearConstraint(np.diag(np.ones(n)), 0, np.inf)
              ])

ret

<ipython-input-113-d1f01677a211>:2: RuntimeWarning: invalid value encountered in log
  return -sum(np.log(P))
<ipython-input-113-d1f01677a211>:2: RuntimeWarning: divide by zero encountered in log
  return -sum(np.log(P))


     fun: 149.7299681309567
     jac: array([-42.86771148, -47.96745501, -46.05907655, -55.24368741,
       -52.67583609, -57.86044033, -43.98209154, -35.93104719,
       -42.64519546, -32.06679176, -27.1765754 , -62.39202436,
       -59.98797919, -30.85140766, -65.80178793, -33.36781001,
       -24.75110808, -40.71472719, -25.15998298, -59.00560874,
       -54.15112082, -60.86818956, -42.75949793, -42.02330352,
       -38.5809788 , -35.29968357, -53.71524224, -22.00067175,
       -46.63981826, -26.90178503, -47.23276674, -31.12588615,
       -55.90116795, -47.56450024, -27.34027326, -59.58581574,
       -20.36955831, -65.78368996, -52.51742571, -55.87628715])
 message: 'Optimization terminated successfully'
    nfev: 156
     nit: 45
    njev: 45
  status: 0
 success: True
       x: array([0.02332479, 0.02085181, 0.02171115, 0.01810101, 0.01898435,
       0.01728502, 0.02273968, 0.02782575, 0.02344591, 0.03118654,
       0.03679643, 0.01602815, 0.01667142, 0.03241758, 0.01519499,
    

In [131]:
boot = ss.rv_discrete(values=(brain, ret.x))

In [135]:
BIG = 1000000

sample = boot.rvs(size=(BIG, n), random_state=38)

In [136]:
sample = sample.mean(axis=1)
sample.size

1000000

In [137]:
np.sum(sample < np.mean(brain)) / BIG

0.019618